In [1]:
import kagglehub
import shutil
import os
import cv2
import json
import numpy as np
from skimage.feature import graycomatrix, graycoprops, hog
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
# Descarga el dataset si no existe en la carpeta data
dataset_final_path = "data/EuroSAT/"

if os.path.exists(dataset_final_path):
    print("El dataset ya está descargado. No es necesario volver a descargarlo")
else:
    print("Descargando el dataset...")
    download_path = kagglehub.dataset_download("apollo2506/eurosat-dataset")
    original_path = os.path.join(download_path, "EuroSAT")

    # Verifica que la carpeta EuroSAT existe en el dataset descargado
    if os.path.exists(original_path):
        # Mueve la carpeta EuroSAT a data
        shutil.move(original_path, dataset_final_path)
        print(f"Dataset movido a: {dataset_final_path}")
    else:
        print("Error: No se encontró la carpeta EuroSAT dentro del dataset")

El dataset ya está descargado. No es necesario volver a descargarlo


In [3]:
# Mapeo de etiquetas desde el fichero label_map.json
label_map_path = os.path.join(dataset_final_path, "label_map.json")

with open(label_map_path, "r") as file:
    label_map = json.load(file)

print("Etiquetas:", label_map)

Etiquetas: {'AnnualCrop': 0, 'Forest': 1, 'HerbaceousVegetation': 2, 'Highway': 3, 'Industrial': 4, 'Pasture': 5, 'PermanentCrop': 6, 'Residential': 7, 'River': 8, 'SeaLake': 9}


In [4]:
# Extracción de características Haralick y HOG de las imágenes

# Ref. https://scikit-image.org/docs/stable/api/skimage.feature.html#skimage.feature.graycomatrix
# Ref. https://scikit-image.org/docs/stable/api/skimage.feature.html#skimage.feature.graycoprops
# Ref. https://medium.com/top-python-libraries/12-examples-of-image-texture-analysis-in-python-6cf7c179ada7
# Ref. https://medium.com/@girishajmera/feature-extraction-of-images-using-glcm-gray-level-cooccurrence-matrix-e4bda8729498
# Ref. https://medium.com/swlh/histogram-of-oriented-gradients-hog-for-multiclass-image-classification-and-image-recommendation-cf0ea2caaae8

# Parámetros para calcular la matriz de co-ocurrencia de niveles de gris (GLCM)
distances = [1]  # Distancia de 1 píxel entre los valores a comparar
angles = [0]  # Ángulo de 0 grados (horizontal) para evaluar la textura

X_haralick, X_hog, X_combined, y = [], [], [], []

# Recorre todas las carpetas dentro del dataset
for class_name, label in label_map.items():

    class_path = os.path.join(dataset_final_path, class_name)

    if not os.path.isdir(class_path):
        continue  # Si no es una carpeta válida se ignora

    print(f"Procesando: {class_name}")

    # Recorre todas las imágenes dentro de la carpeta actual
    for img_name in os.listdir(class_path):
        img_path = os.path.join(class_path, img_name)

        # Carga la imagen en escala de grises
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        # Si la imagen no se puede leer se ignora
        if img is None:
            continue

        # Calcula la matriz de co-ocurrencia de niveles de gris (GLCM)
        glcm = graycomatrix(img, distances, angles, symmetric=True, normed=True)

        # Extracción de características de Haralick
        contrast = graycoprops(glcm, 'contrast')[0, 0]
        dissimilarity = graycoprops(glcm, 'dissimilarity')[0, 0]
        homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
        energy = graycoprops(glcm, 'energy')[0, 0]
        correlation = graycoprops(glcm, 'correlation')[0, 0]
        asm = graycoprops(glcm, 'ASM')[0, 0]
        haralick_features = [contrast, dissimilarity, homogeneity, energy, correlation, asm]

        # Histograma de Gradientes Orientados (HOG)
        hog_features = hog(img, pixels_per_cell=(8, 8), cells_per_block=(2, 2), feature_vector=True)

        # Almaceno las características en listas separadas
        X_haralick.append(haralick_features)
        X_hog.append(list(hog_features))
        X_combined.append(haralick_features + list(hog_features))
        y.append(label)

print("Extracción de características completada")

Procesando: AnnualCrop
Procesando: Forest
Procesando: HerbaceousVegetation
Procesando: Highway
Procesando: Industrial
Procesando: Pasture
Procesando: PermanentCrop
Procesando: Residential
Procesando: River
Procesando: SeaLake
Extracción de características completada


In [5]:
# Convierte los datos en arrays
X_haralick = np.array(X_haralick)
X_hog = np.array(X_hog)
X_combined = np.array(X_combined)
y = np.array(y)

In [6]:
# Función para entrenar y evaluar el modelo con diferentes características
def train_and_evaluate(X, feature_name):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print(f"Entrenando con {feature_name}...")
    svm = SVC(kernel='linear')
    svm.fit(X_train, y_train)
    y_pred = svm.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Precisión con {feature_name}: {accuracy:.2f}\n")

In [7]:
# Evaluar cada extractor de características
train_and_evaluate(X_haralick, "Haralick (GLCM)")
train_and_evaluate(X_hog, "HOG")
train_and_evaluate(X_combined, "Haralick + HOG")

Entrenando con Haralick (GLCM)...
Precisión con Haralick (GLCM): 0.68

Entrenando con HOG...
Precisión con HOG: 0.66

Entrenando con Haralick + HOG...
Precisión con Haralick + HOG: 0.74

